In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=03d68bbb335e3319b642de3b226142c3b9e2e674f035bbba043e56c6f4c21f48
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 3.4MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.04  ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 40.53 , C = 980.40 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_13_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=258541, Fri Apr 17 20:35:06 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.022974983698809e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.017051947031837e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10539.78695381] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10418.79600205] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 9.730653587205139e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   37    703 7.219608598777341e+03 1.1e+00 2.13e+00  2e+00  2e+00 0:03.3
   83   1577 6.613547302437287e+03 1.2e+00 1.63e+00  2e+00  2e+00 0:07.4
  100   1900 5.991388001955180e+03 1.2e+00 1.39e+00  1e+00  1e+00 0:08.9
  168   3192 4.264095662743299e+03 1.2e+00 7.31e-01  7e-01  7e-01 0:14.9
  200   3800 3.601744066170275e+03 1.2e+00 5.26e-01  5e-01  5e-01 0:17.7
  292   5548 2.926302093338021e+03 1.2e+00 1.95e-01  2e-01  2e-01 0:25.8
  300   5700 2.899895657416988e+03 1.2e+00 1.78e-01  2e-01  2e-01 0:26.5
  400   7600 2.818205341186276e+03 1.3e+00 5.98e-02  6e-02  6e-02 0:35.4
  500   9500 2.808332088847650e+03 1.3e+00 2.04e-02  2e-02  2e-02 0:44.2
  600  11400 2.807165093999216e+03 1.3e+00 6.99e-03  7e-03  7e-03 0:53.0
  700  13300 2.807029132554600e+03 1.3e+00 2.26e-03  2e-03  2e-03 1:01.8
  800  15200 2.807014702904255e+03 1.3e+00 8.21e-04  8e-04  8e-04 1:10.6
  900  17100 2.807012853978469e+03 1.3e+00 2.74e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10971.99080306] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.870076313982616e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.000352244341787e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 9.523706672850158e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   37    703 6.934313508607352e+03 1.1e+00 2.00e+00  2e+00  2e+00 0:03.4
   81   1539 6.374046356089609e+03 1.2e+00 1.69e+00  2e+00  2e+00 0:07.4
  100   1900 6.228816170881455e+03 1.2e+00 1.47e+00  1e+00  1e+00 0:09.3
  163   3097 4.439282650441174e+03 1.2e+00 8.35e-01  8e-01  8e-01 0:15.3
  200   3800 3.890942664949524e+03 1.2e+00 5.88e-01  6e-01  6e-01 0:18.9
  284   5396 2.978716366064035e+03 1.2e+00 2.36e-01  2e-01  2e-01 0:26.9
  300   5700 2.938281387701274e+03 1.2e+00 1.95e-01  2e-01  2e-01 0:28.4
  400   7600 2.820130366233173e+03 1.2e+00 6.36e-02  6e-02  6e-02 0:37.6
  500   9500 2.808927322506315e+03 1.2e+00 2.24e-02  2e-02  2e-02 0:46.6
  600  11400 2.807215780621849e+03 1.2e+00 7.70e-03  8e-03  8e-03 0:55.7
  700  13300 2.807037473527946e+03 1.2e+00 2.78e-03  3e-03  3e-03 1:04.8
  800  15200 2.807015528091898e+03 1.3e+00 9.47e-04  9e-04  9e-04 1:14.9
  900  17100 2.807012937179022e+03 1.3e+00 3.17e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10346.84861218] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.300295495962717e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 9.354467309022542e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10308.00882195] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   35    665 5.810260470413866e+03 1.1e+00 1.94e+00  2e+00  2e+00 0:03.4
   77   1463 5.571202559838386e+03 1.2e+00 1.64e+00  2e+00  2e+00 0:07.4
  100   1900 5.204335129135591e+03 1.2e+00 1.38e+00  1e+00  1e+00 0:09.6
  163   3097 4.303142206207818e+03 1.2e+00 8.12e-01  8e-01  8e-01 0:15.6
  200   3800 4.040380888910525e+03 1.2e+00 5.65e-01  6e-01  6e-01 0:19.2
  283   5377 3.733124144277925e+03 1.2e+00 3.08e-01  3e-01  3e-01 0:27.2
  300   5700 3.707267131286003e+03 1.3e+00 2.80e-01  3e-01  3e-01 0:28.9
  400   7600 3.610081352808495e+03 1.3e+00 1.21e-01  1e-01  1e-01 0:38.7
  500   9500 3.585874871631544e+03 1.3e+00 6.67e-02  6e-02  7e-02 0:48.8
  600  11400 3.578480012449295e+03 1.4e+00 4.14e-02  4e-02  4e-02 0:58.5
  700  13300 3.574496953991664e+03 1.5e+00 2.71e-02  3e-02  3e-02 1:08.2
  800  15200 3.572408304760504e+03 1.8e+00 2.12e-02  2e-02  3e-02 1:17.7
  900  17100 3.571368029904116e+03 2.1e+00 1.55e-02  1e-02  2e-02 1:27.4
 1000  19000 3.570918315353907e+03 2.2e+00 9.42e-03

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F13_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:200])
    SVM_Fun [i] = F13(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F13(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2858.18445504]), count=array([1])) 1.7364461204629024e-05
SVM
ModeResult(mode=array([10292.1029062]), count=array([1])) 341.8548625555483
ELN
ModeResult(mode=array([7428.35332638]), count=array([1])) 22.014742383461897
